In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('datasets/saved IMDB dataset.csv')
df.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [3]:
from string import punctuation

In [4]:
punct = punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [5]:
df['Review'] = df['Review'].apply(lambda x: "".join([i for i in x if i not in punct]))

In [6]:
df

,Review,Sentiment
0,Wow Loved this place,1
1,Crust is not good,0
2,Not tasty and the texture was just nasty,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
2743,I just got bored watching Jessice Lange take h...,0
2744,Unfortunately any virtue in this films product...,0
2745,In a word it is embarrassing,0
2746,Exceptionally bad,0


In [7]:
y = df['Sentiment']
y.shape

(2748,)

In [8]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [9]:
text = df["Review"].tolist()
text[:5]

['Wow Loved this place',
 'Crust is not good',
 'Not tasty and the texture was just nasty',
 'Stopped by during the late May bank holiday off Rick Steve recommendation and loved it',
 'The selection on the menu was great and so were the prices']

In [10]:
token = Tokenizer()
token.fit_on_texts(text)

In [11]:
token.word_index

{'the': 1,
 'and': 2,
 'i': 3,
 'a': 4,
 'is': 5,
 'to': 6,
 'it': 7,
 'this': 8,
 'of': 9,
 'was': 10,
 'in': 11,
 'for': 12,
 'not': 13,
 'that': 14,
 'with': 15,
 'my': 16,
 'very': 17,
 'good': 18,
 'on': 19,
 'great': 20,
 'you': 21,
 'but': 22,
 'have': 23,
 'are': 24,
 'movie': 25,
 'as': 26,
 'so': 27,
 'phone': 28,
 'film': 29,
 'its': 30,
 'all': 31,
 'be': 32,
 'one': 33,
 'had': 34,
 '0': 35,
 'at': 36,
 '1': 37,
 'food': 38,
 'like': 39,
 'just': 40,
 'place': 41,
 'time': 42,
 'were': 43,
 'service': 44,
 'an': 45,
 'if': 46,
 'really': 47,
 'from': 48,
 'there': 49,
 'they': 50,
 'bad': 51,
 'we': 52,
 'has': 53,
 'out': 54,
 'well': 55,
 'dont': 56,
 'about': 57,
 'would': 58,
 'no': 59,
 'your': 60,
 'or': 61,
 'by': 62,
 'only': 63,
 'best': 64,
 'ever': 65,
 'even': 66,
 'here': 67,
 'also': 68,
 'back': 69,
 'will': 70,
 'up': 71,
 'when': 72,
 'me': 73,
 'more': 74,
 'than': 75,
 'quality': 76,
 'go': 77,
 'what': 78,
 'love': 79,
 'ive': 80,
 'made': 81,
 'he': 82

In [12]:
vocab_size = len(token.word_index) + 1
vocab_size

5404

In [13]:
encoded_text= token.texts_to_sequences(text)
encoded_text

[[638, 209, 8, 41],
 [1451, 5, 13, 18],
 [13, 400, 2, 1, 1063, 10, 40, 856],
 [1064, 62, 401, 1, 1065, 367, 2210, 2211, 152, 2212, 1066, 1067, 2, 209, 7],
 [1, 432, 19, 1, 294, 10, 20, 2, 27, 43, 1, 433],
 [126, 3, 135, 368, 1452, 2, 3, 185, 16, 639, 640],
 [2213, 7, 121, 316, 14, 283],
 [1,
  1068,
  43,
  39,
  2214,
  2,
  21,
  94,
  369,
  50,
  34,
  103,
  81,
  71,
  2215,
  9,
  42,
  142,
  434,
  511,
  4,
  2216],
 [1, 465, 43, 20, 89],
 [4, 20, 857],
 [44, 10, 17, 1453],
 [58, 13, 77, 69],
 [1,
  1454,
  34,
  59,
  402,
  78,
  27,
  65,
  19,
  78,
  3,
  34,
  6,
  131,
  7,
  127,
  731,
  71,
  142,
  2217,
  512],
 [3, 295, 1, 2218, 2219, 2220, 2221, 2222],
 [3, 10, 2223, 85, 3, 10, 123, 284, 14, 10, 513, 858],
 [3, 10, 2224, 85, 59, 1455, 2225, 2226, 63],
 [200, 435],
 [466, 10, 4, 136, 254, 11, 44],
 [8, 41, 5, 13, 176, 60, 42, 514, 1069, 210],
 [106, 13, 39, 36, 31],
 [1, 2227, 1456],
 [1, 38, 153],
 [44, 5, 68, 859],
 [3, 94, 402, 403, 1, 2228, 5, 40, 270],
 [27,

In [14]:
#getting maximum length of array indexes of encoded_text

max_ = []
def get_max_len(x):
    for i in x[:]:
        max_.append(len(i))
    
    length = np.max(max_)
    return length

max_length = get_max_len(encoded_text)
max_length

1384

In [15]:
maxlen = max_length + 1  
maxlen

1385

In [16]:
X = pad_sequences(encoded_text, padding='pre', maxlen=maxlen)
X

array([[   0,    0,    0, ...,  209,    8,   41],
       [   0,    0,    0, ...,    5,   13,   18],
       [   0,    0,    0, ...,   10,   40,  856],
       ...,
       [   0,    0,    0, ...,    7,    5,  821],
       [   0,    0,    0, ...,    0, 5403,   51],
       [   0,    0,    0, ...,  151,    9,  160]])

In [17]:
X[0].shape

(1385,)

In [18]:
vec_size = 100
#changing each word in an index into a vector of size 200

In [20]:
from tensorflow.keras.optimizers import Adam

In [21]:
model = Sequential()
model.add(Embedding(vocab_size, vec_size, input_length = maxlen))
model.add(LSTM(100))
model.add(Dense(1, kernel_initializer = 'glorot_uniform', activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = Adam(0.0001), metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1385, 100)         540400    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 620,901
Trainable params: 620,901
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
model.fit(X, y, validation_split= 0.2, epochs = 10, batch_size = 15)

Epoch 1/10
147/147 [==============================] - 76s 514ms/step - loss: 0.6914 - accuracy: 0.5446 - val_loss: 0.6918 - val_accuracy: 0.5436
Epoch 2/10
147/147 [==============================] - 80s 545ms/step - loss: 0.6807 - accuracy: 0.6711 - val_loss: 0.6862 - val_accuracy: 0.5600
Epoch 3/10
147/147 [==============================] - 81s 551ms/step - loss: 0.6092 - accuracy: 0.7452 - val_loss: 0.6558 - val_accuracy: 0.6345
Epoch 4/10
147/147 [==============================] - 81s 550ms/step - loss: 0.5163 - accuracy: 0.8403 - val_loss: 0.6419 - val_accuracy: 0.6691
Epoch 5/10
147/147 [==============================] - 81s 549ms/step - loss: 0.4076 - accuracy: 0.8685 - val_loss: 0.6211 - val_accuracy: 0.6745
Epoch 6/10
147/147 [==============================] - 81s 550ms/step - loss: 0.3253 - accuracy: 0.9076 - val_loss: 0.6262 - val_accuracy: 0.7091
Epoch 7/10
147/147 [==============================] - 81s 550ms/step - loss: 0.2599 - accuracy: 0.9354 - val_loss: 0.6996 - val_ac